# SQLDatabaseSequentialChain - Mondial FK - GPT 3.5

In [1]:
!pip install langchain_experimental


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.sql_database import SQLDatabase
from langchain.chains import LLMChain
from langchain_experimental.sql.base import SQLDatabaseSequentialChain
from langchain.llms import OpenAI
from urllib.parse import quote  
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import get_openai_callback
import time
from dotenv import load_dotenv
import os
import sys
import json
load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils

# Schema

In [3]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/11_sql_sequential_queries_chatgpt_{SCHEMA}_fk.json"


In [4]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 
        
def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)

    queries = data["queries"]
    return queries


## Conexão

In [ ]:
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)

db_connection

### Utilizando o SQLDatabase para pegar todas as informações do database

In [6]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables)
db.get_table_names()

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

In [7]:
len(db.get_table_names())

46

## Criando o SQLChainPrompt especifico para o SQLDatabaseSequential

In [8]:
f = open(f"prompts/sql_chain_prompt_template.txt", "r")
prompt_template = f.read()
f.close()


QUERY_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"], template=prompt_template
)

print(QUERY_PROMPT)

input_variables=['input', 'table_info', 'top_k'] output_parser=None partial_variables={} template="Given an input question, first create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, don't limits your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use the following tables:\n{tabl

## Criando o Chain para gerar SQL

In [ ]:
llm = OpenAI(temperature=0,model_name='gpt-3.5-turbo',verbose=True)

db_chain = SQLDatabaseSequentialChain.from_llm(llm, db.db, query_prompt=QUERY_PROMPT, verbose=True, use_query_checker=True, return_intermediate_steps=False, return_sql=True)
db_chain

In [10]:
# with get_openai_callback() as cb:
#     sql_query = db_chain({
#         "query": "What is the area of Thailand?"
#     })
#     print(f"Total Tokens: {cb.total_tokens}")
#     print(f"Prompt Tokens: {cb.prompt_tokens}")
#     print(f"Completion Tokens: {cb.completion_tokens}")
#     print(f"Total Cost (USD): ${cb.total_cost}")
    

# sql_query = sql_query['result']

# print(sql_query)

## Preparando as consultas em linguagem natural para rodar no LLM

In [11]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

# Rodando as consultas no LLM para gerar SQL

In [16]:

# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
number_of_queries_to_delay = 20
count = 0
for instance in queries:
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(5)
        
    with get_openai_callback() as cb:
        start_time = time.time()
        sql_query = db_chain({
            "query": instance["question"]
        })
        end_time = time.time()
        instance["query_string"] = sql_query['result']
        instance['total_tokens'] = cb.total_tokens
        instance['prompt_tokens'] = cb.prompt_tokens
        instance['completion_tokens'] = cb.completion_tokens
        instance['total_cost'] = cb.total_cost  
        instance['time'] = end_time - start_time
        print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
    save_queries(queries)

    count += 1
    
queries



> Entering new SQLDatabaseSequentialChain chain...


c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['country']

> Entering new SQLDatabaseChain chain...
What is the area of Thailand?
SQLQuery:

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:307: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables



> Finished chain.

> Finished chain.
1 What is the area of Thailand? SELECT area
FROM country
WHERE name = 'Thailand' 2.4537250995635986 0.0007509999999999999


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['province']

> Entering new SQLDatabaseChain chain...
What are the provinces with an area greater than 10000?
SQLQuery:
> Finished chain.

> Finished chain.
2 What are the provinces with an area greater than 10000? SELECT name, area
FROM province
WHERE area > 10000 1.559727668762207 0.000798


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['country', 'language']

> Entering new SQLDatabaseChain chain...
What are the languages spoken in Poland?
SQLQuery:
> Finished chain.

> Finished chain.
3 What are the languages spoken in Poland? SELECT name
FROM language
WHERE country = 'POL' 1.3853554725646973 0.0008745000000000001


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['lake']

> Entering new SQLDatabaseChain

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Table names to use:
['city', 'country']

> Entering new SQLDatabaseChain chain...
What is the capital of Georgia?
SQLQuery:
> Finished chain.

> Finished chain.
10 What is the capital of Georgia? SELECT capital
FROM country
WHERE name = 'Georgia' 605.1658601760864 0.000996


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['province']

> Entering new SQLDatabaseChain chain...
How many provinces have areas greater than 1000 in Niger?
SQLQuery:
> Finished chain.

> Finished chain.
11 How many provinces have areas greater than 1000 in Niger? SELECT COUNT(*) 
FROM province 
WHERE country = 'NER' AND area > 1000 1.4739091396331787 0.000808


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['country']

> Entering new SQLDatabaseChain chain...
Tell me about the country with the area largest.
SQLQuery:
> Finished chain.

> Finished chain.
12 Tell me about the country with the area largest. SELECT name, area 
FROM country 
ORDER BY area DESC 
FETCH 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).



> Finished chain.

> Finished chain.
81 What is the total area of the provinces in Canada? SELECT SUM(area) FROM province WHERE country = 'CAN' 605.6764469146729 0.0012965


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['river']

> Entering new SQLDatabaseChain chain...
What is the length of the Tigres River?
SQLQuery:
> Finished chain.

> Finished chain.
82 What is the length of the Tigres River? SELECT length
FROM river
WHERE name = 'Tigres' 2.341500759124756 0.0013155000000000003


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['airport']

> Entering new SQLDatabaseChain chain...
List the Airports with elevation more than 1000
SQLQuery:
> Finished chain.

> Finished chain.
83 List the Airports with elevation more than 1000 SELECT name, elevation
FROM airport
WHERE elevation > 1000 2.08807373046875 0.0009570000000000001


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['airport']

> Entering new SQLDatabaseCh

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area\nFROM country\nWHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 496,
  'prompt_tokens': 482,
  'completion_tokens': 14,
  'total_cost': 0.0007509999999999999,
  'time': 2.4537250995635986},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name, area\nFROM province\nWHERE area > 10000',
  'type': 'simple',
  'total_tokens': 527,
  'prompt_tokens': 512,
  'completion_tokens': 15,
  'total_cost': 0.000798,
  'time': 1.559727668762207},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM language\nWHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 578,
  'prompt_tokens': 563,
  'completion_tokens': 15,
  'total_cost': 0.0008745000000000001,
  'time': 1.3853554725646973},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\n

In [14]:
# # A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
# number_of_queries_to_delay = 5
# count = 0
# for i in range(17, len(queries)):
#     instance = queries[i]
#     if count == number_of_queries_to_delay:
#         count = 0
#         time.sleep(30)
        
#     with get_openai_callback() as cb:
#         start_time = time.time()
#         sql_query = db_chain({
#             "query": instance["question"]
#         })
#         end_time = time.time()
#         instance["query_string"] = sql_query['result']
#         instance['total_tokens'] = cb.total_tokens
#         instance['prompt_tokens'] = cb.prompt_tokens
#         instance['completion_tokens'] = cb.completion_tokens
#         instance['total_cost'] = cb.total_cost  
#         instance['time'] = end_time - start_time
#         print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
#     save_queries(queries)
#     count += 1
    
# queries



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['politics']

> Entering new SQLDatabaseChain chain...
Show all the different types of government.
SQLQuery:
> Finished chain.

> Finished chain.
18 Show all the different types of government. SELECT DISTINCT government FROM politics


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['country', 'province', 'provincelocalname', 'provinceothername']

> Entering new SQLDatabaseChain chain...
List all provinces in Germany
SQLQuery:
> Finished chain.

> Finished chain.
19 List all provinces in Germany SELECT name
FROM province
WHERE country = 'DEU'


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['ethnicgroup', 'country', 'countrypops']

> Entering new SQLDatabaseChain chain...
What is the percentage of mulatto ethnicity in Brazil?
SQLQuery:
> Finished chain.

> Finished chain.
20 What is the percentage of mulatto ethnicity in Brazil? SELECT percentage
FROM ethnicgroup
WH

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 492,
  'prompt_tokens': 480,
  'completion_tokens': 12,
  'total_cost': 0.000744},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name, country, area\nFROM province\nWHERE area > 10000\nLIMIT 5;',
  'type': 'simple',
  'total_tokens': 532,
  'prompt_tokens': 510,
  'completion_tokens': 22,
  'total_cost': 0.000809},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM language\nWHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 576,
  'prompt_tokens': 561,
  'completion_tokens': 15,
  'total_cost': 0.0008715},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\nFROM lake\nWHERE name = 'Lake Kariba'",
  'type': 'simple',
  'total_tokens': 618,
  'prompt_toke

In [15]:
queries = read_queries()
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 492,
  'prompt_tokens': 480,
  'completion_tokens': 12,
  'total_cost': 0.000744},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name, country, area\nFROM province\nWHERE area > 10000\nLIMIT 5;',
  'type': 'simple',
  'total_tokens': 532,
  'prompt_tokens': 510,
  'completion_tokens': 22,
  'total_cost': 0.000809},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM language\nWHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 576,
  'prompt_tokens': 561,
  'completion_tokens': 15,
  'total_cost': 0.0008715},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\nFROM lake\nWHERE name = 'Lake Kariba'",
  'type': 'simple',
  'total_tokens': 618,
  'prompt_toke

#### Fixing queries

In [12]:
to_fix = [40,59,62,72,85,99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    with get_openai_callback() as cb:
        start_time = time.time()
        sql_query = db_chain({
            "query": instance["question"]
        })
        end_time = time.time()
        instance["query_string"] = sql_query['result']
        instance['total_tokens'] = cb.total_tokens
        instance['prompt_tokens'] = cb.prompt_tokens
        instance['completion_tokens'] = cb.completion_tokens
        instance['total_cost'] = cb.total_cost  
        instance['time'] = end_time - start_time
        q[pos] = instance
        print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
        save_queries(q)



> Entering new SQLDatabaseSequentialChain chain...


c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['borders', 'country', 'geo_sea']

> Entering new SQLDatabaseChain chain...
How many countries that are close to the Mediterranean Sea?
SQLQuery:

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:307: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables



> Finished chain.

> Finished chain.
41 How many countries that are close to the Mediterranean Sea? SELECT COUNT(DISTINCT country.code)
FROM country
JOIN borders ON country.code = borders.country1 OR country.code = borders.country2
JOIN geo_sea ON country.code = geo_sea.country
WHERE geo_sea.sea = 'Mediterranean Sea' 2.48130202293396 0.0011865


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['city', 'organization']

> Entering new SQLDatabaseChain chain...
 List the names of capital cities which are the base for organizations in alphabetical order
SQLQuery:
> Finished chain.

> Finished chain.
60  List the names of capital cities which are the base for organizations in alphabetical order SELECT c.name
FROM city c
JOIN organization o ON c.name = o.city
WHERE c.province = 'Capital'
ORDER BY c.name ASC 1.9335837364196777 0.001126


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['country', 'geo_sea']

> Entering new SQLDatabaseChain chain.